# GROUP 24 - TASK 8 

### Install Transformers Library

In [6]:
pip install transformers -U

     |████████████████████████████████| 2.2MB 18.7MB/s 
     |████████████████████████████████| 3.3MB 50.7MB/s 
     |████████████████████████████████| 870kB 52.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=898d8f7c418393cca88c0db5a7ec7acf27ba90857b5fa5352beee6e8ec418460
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [7]:
pip install watchdog==0.8.3

     |████████████████████████████████| 92kB 8.5MB/s 
  Created wheel for watchdog: filename=watchdog-0.8.3-cp37-none-any.whl size=72888 sha256=a52ad4e750be534c4aeb85bc61e4b11d398e8f7bc5bab8ebdb95b9ecd3df54ea
  Stored in directory: /root/.cache/pip/wheels/3b/a0/6f/8c52fea32c994f833b55e7b73078f81c155ec108011c04e708
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=e37a6eb6d0af25209c67a95c98f8f0290f472a6da11fbe09e851112c1631df3d
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog pathtools


In [8]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 18.2MB/s 
     |████████████████████████████████| 1.2MB 34.1MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 194kB 55.3MB/s 
     |████████████████████████████████| 8.2MB 51.1MB/s 
     |████████████████████████████████| 122kB 57.1MB/s 
     |████████████████████████████████| 2.1MB 42.8MB/s 
     |████████████████████████████████| 112kB 47.3MB/s 
     |████████████████████████████████| 245kB 61.8MB/s 
     |████████████████████████████████| 4.6MB 44.3MB/s 
     |████████████████████████████████| 163kB 56.1MB/s 
     |████████████████████████████████| 112kB 56.9MB/s 
     |████████████████████████████████| 133kB 55.0MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
     |████████████████████████████████| 122kB 53.2MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-c

### Define functions Donwload Measeval data

In [9]:
#downloading the data files from the github repository off the competetion
!git clone https://github.com/harperco/MeasEval.git

Cloning into 'MeasEval'...
remote: Enumerating objects: 3655, done.
remote: Counting objects: 100% (3655/3655), done.
remote: Compressing objects: 100% (2945/2945), done.
remote: Total 3655 (delta 675), reused 3611 (delta 632), pack-reused 0
Receiving objects: 100% (3655/3655), 1.33 MiB | 22.61 MiB/s, done.
Resolving deltas: 100% (675/675), done.


In [10]:

#importing necessary packages
import os
from os import walk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pandas as pd
import string
from sklearn.model_selection import train_test_split
import shutil
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
def getfile_bases(folder):
  ## read textfilenames
  txtnames = []
  textpath = ''
  if folder == 'eval':
    textpath = '/content/MeasEval/data/'+folder+'/text/'
  else:
    textpath = '/content/MeasEval/data/'+folder+'/text/'

  for (dirpath, dirnames, filenames) in walk(textpath):
      txtnames.extend(filenames)
      break
  ## read tsvfilenames
  tsvnames = []
  for (dirpath, dirnames, filenames) in walk('/content/MeasEval/data/'+folder+'/tsv/'):
      tsvnames.extend(filenames)
      break
  ## list the mathced ones (having both tsv and txt file)
  b = []
  for f in filenames:
    base, ext = os.path.splitext(f)
    if base + '.tsv' in tsvnames :
      b.append(base)
  return b;

In [12]:
def read_text_files(folder,base_list):
  textdata = []
  textpath = ''
  if folder == 'eval':
    textpath = '/content/MeasEval/data/'+folder+'/text/'
  else:
    textpath = '/content/MeasEval/data/'+folder+'/text/'

  for base in base_list:
    f = open(textpath + base + '.txt', "r")
    text = f.read()
    f.close()
    textdata.append((base,text))
  X = pd.DataFrame(textdata, columns = ['docId', 'inputText'])
  return X

In [13]:
def read_tsv_files(folder,base_list):
  dataframes = []
  for base in base_list:
    df = pd.read_csv('/content/MeasEval/data/'+folder+'/tsv/' + base + '.tsv',sep="\t")
    dataframes.append(df)
  Y = pd.concat(dataframes, ignore_index=True)
  return Y

In [14]:
## removes the punctuations from words
def process_sentence(sentence):
    words = sentence.split()
    words2 = word_tokenize(sentence)
    removelist = []
    for w in words2:
        if w in string.punctuation:
            removelist.append(w)
    for i in removelist:
      words2.remove(i)
    return words

In [15]:
## find annottype of words w.r.t its index
def find_annotations(words_indices,labels):
  matched = [] 
  for i in words_indices:
    word = i[0]
    index = i[1]
    annotType = None
    for index2, row2 in labels.iterrows():
      start = row2['startOffset']
      end = row2['endOffset']
      if start <= index <= end:
        annotType = row2['annotType']
        break
    if annotType is None:
      matched.append('O')
    else:
      matched.append(annotType)
  return matched

### Create Sentence Tags and filed

In [16]:
## find unique indices for each word
def find_indices2(inputText, words):
  words_with_indices = []
  startlist = []
  first = 1
  sentencestart = 0
  end = 0
  for word in words:
    x = inputText.find(word) 
    if x != -1:
      if first == 1:
        sentencestart = x
        first = 0
      start = x
      end =  x + len(word)
      tmp = '!' * len(word)
      inputText = inputText[:start] + tmp + inputText[end:]
      startlist.append(str(start))
      words_with_indices.append((word,start,start+len(word)))
  
  return sentencestart,end,startlist,words_with_indices,inputText

In [17]:
#concattanating all the bases
bases_train = getfile_bases('train')
bases_eval = getfile_bases('eval')

In [18]:
#reading tsv files
Y_trainf = read_tsv_files('train',bases_train)
Y_evalf = read_tsv_files('eval',bases_eval)
Y = pd.concat([Y_trainf,Y_evalf])

In [19]:
# A few imports and set up our paths
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding

trainpaths = ["/content/MeasEval/data/eval/tsv/",
             "/content/MeasEval/data/train/tsv/"]

textpaths = ["/content/MeasEval/data/eval/text/",
            "/content/MeasEval/data/train/text/"]

In [20]:
# Collect all the ids and all the text files in both the train and trial directories
# Set our train test split for doing initial model development.
docIds = []
textset = {}
for fileset in textpaths:
    for fn in os.listdir(fileset):
        with open(fileset+fn) as textfile:
            text = textfile.read() #.splitlines()
            #print(fn[:-4])
            textset[fn[:-4]] = text
            docIds.append(fn[:-4])

random.seed(42)
random.shuffle(docIds)

trainIds = docIds[:248]
testIds = docIds[248:]

In [21]:
#Fincding indices and annotations and getting them together
trainsentenceList = []
traintagList = []
trainstartList = []
testsentenceList = []
testtagList = []
teststartList = []
testdocIds = []
trainwordList = []
trainannotList = []
trainsentenceIdList = []
testwordList = []
testannotList = []
testsentenceIdList = []
sentenceindexList = []
i = 0
j = 0
for fileset in trainpaths:
    for fn in os.listdir(fileset):
        with open(fileset+fn) as annotfile:
            inputText = textset[fn[:-4]]
            next(annotfile)
            labels = Y.loc[Y.docId == fn[:-4]]


            if fn[:-4] in trainIds:
              sentences = sent_tokenize(inputText)
              for sentence in sentences:
                words = process_sentence(sentence)
                sentence = ' '.join(words);
                if sentence: # if sentence is not empty
                  trainsentenceList.append(' '.join(words))
                  Sstart,Send,start,words_indices,inputText = find_indices2(inputText,words)
                  annotations = find_annotations(words_indices,labels)
                  traintagList.append(' '.join(annotations))       
                  trainstartList.append(' '.join(start))
                  sentenceids = [i] * (len(words))
                  trainwordList.extend(words)
                  trainannotList.extend(annotations)
                  trainsentenceIdList.extend(sentenceids)
                  sentenceindexList.append((fn[:-4],Sstart,Send))
                  i+=1
            else:
              sentences = sent_tokenize(inputText)
              for sentence in sentences:
                words = process_sentence(sentence)
                sentence = ' '.join(words);
                if sentence: # if sentence is not empty
                  testsentenceList.append(' '.join(words))
                  Sstart,Send,start,words_indices,inputText = find_indices2(inputText,words)
                  annotations = find_annotations(words_indices,labels)
                  testtagList.append(' '.join(annotations))
                  teststartList.append(' '.join(start))
                  testdocIds.append(fn[:-4]) 
                  sentenceids = [j] * (len(words))
                  testwordList.extend(words)
                  testannotList.extend(annotations)
                  testsentenceIdList.extend(sentenceids)
                  sentenceindexList.append((fn[:-4],Sstart,Send))
                  j+=1   


In [22]:
df = pd.DataFrame(sentenceindexList, columns=['docId','sentenceStart','sentenceEnd'])

In [23]:
# To view sentence index list generated from above piece of code
display(df[:10])


,docId,sentenceStart,sentenceEnd
0,S2213671113000908-643,0,91
1,S2213671113000908-643,92,181
2,S0019103512003995-2096,0,228
3,S0019103512003995-2096,229,522
4,S0019103512003995-2096,523,555
5,S0019103512003995-2096,556,747
6,S0019103512003995-2096,748,985
7,S0019103512003995-2096,986,1092
8,S1359645413009816-2227,0,106
9,S1359645413009816-2227,107,184


In [24]:
# creating panda Data frame of training data
train_data = pd.DataFrame({'sentence_id':trainsentenceIdList, 'words':trainwordList,'labels':trainannotList})

In [25]:
# to view a sample of training data 
print(train_data[100:130])

     sentence_id        words            labels
100            3        2003;                 O
101            3  Ben-Jaffel,                 O
102            3        2007,                 O
103            3        2008)                 O
104            3         (see                 O
105            3        Figs.                 O
106            4            5                 O
107            4          and                 O
108            4            6                 O
109            4           of                 O
110            4          K10                 O
111            4          for                 O
112            4          the                 O
113            4    results).                 O
114            5          The                 O
115            5      results                 O
116            5        imply                 O
117            5         that                 O
118            5          the                 O
119            5        lower  MeasuredP

In [26]:
# creating panda Data frame of training data
test_data = pd.DataFrame({'sentence_id':testsentenceIdList, 'words':testwordList,'labels':testannotList})

In [27]:
# removing some uneccesary directories
try:
  shutil.rmtree('/content/cache_dir')
except:
  pass

## TRAINING MODEL


In [28]:
# Importing header files to Train and Test model
import numpy as np
import pandas as pd
from scipy.special import softmax
from simpletransformers.ner import NERModel,NERArgs

In [29]:
# get panda dataframe labels as a python list
label = train_data["labels"].unique().tolist()

###MODEL HYPERPARAMETERS

In [30]:
args = NERArgs()
args.num_train_epochs = 7
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.reprocess_input_data = True

Training the NERModel using Sci-Bert as encoder.
SciBERT is a BERT model trained on scientific text.
SciBERT has its own vocabulary (scivocab) that's built to best match the training corpus.

In [31]:
model = NERModel("bert", 'allenai/scibert_scivocab_uncased',labels=label, args=args, use_cuda=True)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [32]:
# Training the model using the train data from github repository
model.train_model(train_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 7:   0%|          | 0/40 [00:00<?, ?it/s]

...

Running Epoch 3 of 7:   0%|          | 0/40 [00:00<?, ?it/s]

Running Epoch 4 of 7:   0%|          | 0/40 [00:00<?, ?it/s]

Running Epoch 5 of 7:   0%|          | 0/40 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/40 [00:00<?, ?it/s]

(280, 0.14522306696120982)

In [33]:
# Storing result after model evaluation on Test Data
result, model_outputs, predictions = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/29 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Quantity seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MeasuredEntity seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Qualifier seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: MeasuredProperty seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [34]:
# Displaying results
print(result)

{'eval_loss': 0.4081637815668665, 'precision': 0.5252206809583859, 'recall': 0.6023138105567607, 'f1_score': 0.5611316941731223}


### Create ents{} from prediction

In [35]:
# to get entities from the prediction
ents = {}
counts = { "total": 0, "Quantity": 0, "MeasuredEntity": 0, "MeasuredProperty": 0, "Qualifier": 0}
for docId in testdocIds:
  ents[docId] = {}
  for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
    ents[docId][entType] = []

In [36]:
# to get counts of different type of entities ( code from original measeval repo )
for docId, line,starts,sentence in zip(testdocIds,predictions,teststartList,testsentenceList):
    text = textset[docId]
    counts["total"] += 1
    for tag,start,word in zip(line,starts.split(),sentence.split()):
      for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
          if tag == entType:
            ents[docId][entType].append((word,int(start),int(int(start) + len(word))))
            counts[entType]+=1
            counts["total"] += 1

In [37]:
# to get an idea of what are the predicted labels produced by our model on test data
for docIds in testdocIds:
  print(docIds)
  print(ents[docIds]['Quantity'])
  print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
  print(ents[docIds]['MeasuredEntity'])
  print("------------------------------------------------------")
  print(ents[docIds]['MeasuredProperty'])
  print("++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")

Streaming output truncated to the last 5000 lines.
------------------------------------------------------
[]
++++++++++++++++++++++++++++++++++++++++++++++++++++++


S0012821X12004384-952
[('<30%', 249, 253), ('>30%', 380, 384), ('2614.7', 658, 664), ('and', 665, 668), ('2619.6', 669, 675), ('m', 676, 677)]
++++++++++++++++++++++++++++++++++++++++++++++++++++++
[('samples', 236, 243), ('wood/plant', 254, 264), ('tissue', 265, 271), ('samples', 367, 374), ('wood/plant', 385, 395), ('tissue.', 396, 403), ('Values', 641, 647), ('shaded', 648, 654)]
------------------------------------------------------
[]
++++++++++++++++++++++++++++++++++++++++++++++++++++++


S0012821X12004384-952
[('<30%', 249, 253), ('>30%', 380, 384), ('2614.7', 658, 664), ('and', 665, 668), ('2619.6', 669, 675), ('m', 676, 677)]
++++++++++++++++++++++++++++++++++++++++++++++++++++++
[('samples', 236, 243), ('wood/plant', 254, 264), ('tissue', 265, 271), ('samples', 367, 374), ('wood/plant', 385, 395), ('tissue.', 39

### Entities to TSV - Modifying baseline (Some parts sourced from measeval original repo) 

In [38]:
# to get entities from the prediction
ents = {}
counts = { "total": 0, "Quantity": 0, "MeasuredEntity": 0, "MeasuredProperty": 0, "Qualifier": 0}
for docId in testdocIds:
  ents[docId] = {}
  for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
    ents[docId][entType] = []

In [39]:
# Set shorthands for annotation spans
typemap = {"Quantity": "QUANT",
           "MeasuredEntity": "ME", 
           "MeasuredProperty": "MP", 
           "Qualifier": "QUAL"}

In [40]:
# We have to train each model separately, because spacy doesn't let us have Multiple entities that overlap,
# Note that we have data split for train / test, and we also have full training data.

trainents = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
traindata = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
testents = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
testdata = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}

alltrainents = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
alltraindata = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}

for fileset in trainpaths:
    for fn in os.listdir(fileset):
        entities = {"QUANT": [], "ME": [], "MP": [], "QUAL": []}
        with open(fileset+fn) as annotfile:
            text = textset[fn[:-4]]
            next(annotfile)
            annots = annotfile.read().splitlines()
            for a in annots:
                annot = a.split("\t")
                atype = typemap[annot[2]]
                start = int(annot[3])
                stop = int(annot[4])
                # This is where we toss out the overlaps:
                overlap = False
                for ent in entities[atype]:
                    if ((start >= ent[0] and start <= ent[1]) or (stop >= ent[0] and stop <= ent[1]) or
                        (ent[0] >= start and ent[0] <= stop) or (ent[1] >= start and ent[1] <= stop)):
                        #print(str(start)+"-"+str(stop)+" overlaps " + str(ent))
                        overlap = True
                if overlap == False:    
                    entities[atype].append((start, stop, atype))
                    if fn[:-4] in testIds:
                      ents[fn[:-4]][annot[2]].append((text[start:stop],start,stop))
            if fn[:-4] in trainIds:
                traindata["QUANT"].append((text, {"entities": entities["QUANT"]}))
                traindata["ME"].append((text, {"entities": entities["ME"]}))
                traindata["MP"].append((text, {"entities": entities["MP"]}))
                traindata["QUAL"].append((text, {"entities": entities["QUAL"]}))
                trainents["QUANT"].extend(entities["QUANT"])
                trainents["ME"].extend(entities["ME"])
                trainents["MP"].extend(entities["MP"])
                trainents["QUAL"].extend(entities["QUAL"])
            else:
                testdata["QUANT"].append((text, {"entities": entities["QUANT"]}))
                testdata["ME"].append((text, {"entities": entities["ME"]}))
                testdata["MP"].append((text, {"entities": entities["MP"]}))
                testdata["QUAL"].append((text, {"entities": entities["QUAL"]}))
                testents["QUANT"].extend(entities["QUANT"])
                testents["ME"].extend(entities["ME"])
                testents["MP"].extend(entities["MP"])
                testents["QUAL"].extend(entities["QUAL"])
            alltraindata["QUANT"].append((text, {"entities": entities["QUANT"]}))
            alltraindata["ME"].append((text, {"entities": entities["ME"]}))
            alltraindata["MP"].append((text, {"entities": entities["MP"]}))
            alltraindata["QUAL"].append((text, {"entities": entities["QUAL"]}))
            alltrainents["QUANT"].extend(entities["QUANT"])
            alltrainents["ME"].extend(entities["ME"])
            alltrainents["MP"].extend(entities["MP"])
            alltrainents["QUAL"].extend(entities["QUAL"])

In [41]:
# Sample ents of the doc
ents['S2213158213000582-751']

{'MeasuredEntity': [('activity', 33, 41), ('FDR', 93, 96), ('FDR', 191, 194)],
 'MeasuredProperty': [('q', 97, 98), ('q', 195, 196)],
 'Qualifier': [],
 'Quantity': [('50 ms', 27, 32), ('0.05', 101, 105), ('0.05', 199, 203)]}

In [42]:
#process entities
for docId, allents in ents.items():
  for entType in ["Quantity", "MeasuredEntity", "MeasuredProperty", "Qualifier"]:
    tmp = allents[entType]
    removelist = []
    for i in range(len(tmp)-1):
      if tmp[i+1][1] - tmp[i][2] < 2:
        tmp[i+1] = (tmp[i][0] + ' ' + tmp[i+1][0],tmp[i][1], tmp[i+1][2])
        removelist.append(i)
    for i in sorted(removelist, reverse=True):
      tmp.pop(i)

In [43]:
# A few imports and set up our paths
import itertools
import random
import os
from spacy.util import minibatch, compounding

trainpaths = ["/content/MeasEval/data/eval/tsv/",
             "/content/MeasEval/data/train/tsv/"]

textpaths = ["/content/MeasEval/data/eval/text/",
            "/content/MeasEval/data/train/text/"]

In [44]:
# code from original measeval to Attempting to identify a unit
import json
units = []

for fileset in trainpaths:
    for fn in os.listdir(fileset):
        # Let's make sure to limit the units to just the smaller train set
        if fn[:-4] in trainIds:
            with open(fileset+fn) as annotfile:
                text = textset[fn[:-4]]
                next(annotfile)
                annots = annotfile.read().splitlines()
                for a in annots:
                    annot = a.split("\t")
                    atype = typemap[annot[2]]
                    if atype == "QUANT" and annot[7] != "":
                        jsondata = json.loads(annot[7])
                        if "unit" in jsondata:
                            units.append(jsondata["unit"])
uniqunits = list(set(units))

In [45]:
%mkdir /content/MeasEval/baseline-simpler-split

In [46]:
# This last, fairly unweildy chunk of code is:
# * collecting everything, 
# * Building the TSV strings
# * Attempting to identify a unit
# * matching and populating annotSet based on knockout logic, 
# * resorting, and populating TSV files.

# Configure header string and submission directory (latter needs to exist.)
header = "docId\tannotSet\tannotType\tstartOffset\tendOffset\tannotId\ttext\tother"
subdir = "/content/MeasEval/baseline-simpler-split/"

for docId, allents in ents.items():
    # First we collect our Quantities
    # We want to get the string version, the full set, and the "knockout" list.
    quantstrings = []
    quants = []
    knockout = []
    annotSet = 1
    for quant in allents['Quantity']:
        # Match units in the Quant, then find the longest unit 
        unitmatches = []
        for unit in uniqunits: 
            if unit in quant[0]:
                unitmatches.append(unit)
        if len(unitmatches) > 0: 
            unit = max(unitmatches, key=len)
        # Build the quantity string, and alsedso the dictionary for quant and knockout.
        quantstrings.append(docId + "\t" + str(annotSet) + "\tQuantity\t" + str(quant[1]) + "\t" +
                          str(quant[2]) + "\t1\t" + quant[0]+"\t{\"unit\": \"" + unit +  "\"}")
        quants.append({"annotSet": annotSet, "annotId": 1, "start": quant[1], "end": quant[2], 
                       "text": quant[0], "type": "Quantity"}) 
        knockout.append({"annotSet": annotSet, "annotId": 1, "start": quant[1], "end": quant[2], 
                       "text": quant[0], "type": "Quantity"}) 
        annotSet+=1
    
    # So now we want to do the ents, as we need this queued up to do more matching with the MPs
    mestrings = []
    mestring = ""
    mes = []
    knockoutmes = []
    #annotSet = 1
    for me in allents['MeasuredEntity']:
        knockoutmes.append({"start": me[1], "end": me[2], "text": me[0], "type": "MeasuredEntity"}) 

    # Now we work through our measured properties.
    mpstrings = []
    mpstring = ""
    mps = []
    knockoutmps = []
    for mp in allents['MeasuredProperty']:
        if len(knockout) > 0 and len(knockoutmes) > 0:
            start = mp[1]
            end = mp[2]
            nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000}
            index = 0
            for q in knockout:
                dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
                mindist = min(dists)
                if mindist < nearest["dist"]:
                    nearest["dist"] = mindist
                    nearest["set"] = q["annotSet"]
                    nearest["id"] = q["annotId"]
                    nearest["index"] = index
                index+=1
            knockout.pop(nearest["index"])

            mpString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredProperty\t" + str(mp[1]) + "\t" + 
                        str(mp[2]) + "\t" + str(nearest["id"]+1) + "\t" + mp[0] + "\t{\"HasQuantity\": \"" + 
                        str(nearest["id"]) + "\"}" )
            mpstrings.append(mpString)
            mps.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": mp[1], 
                        "end": mp[2], "text": mp[0], "type": "MeasuredProperty"})
            knockoutmps.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": mp[1], 
                        "end": mp[2], "text": mp[0], "type": "MeasuredProperty"})


    # Now we do any leftover MEs, which should go straight to a Quantity:

    for me in knockoutmes:
        start = me["start"]
        end = me["end"]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000, "type": ""}
        index = 0                
        for q in knockout:
            dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
            mindist = min(dists)
            if mindist < nearest["dist"]:
                nearest["dist"] = mindist
                nearest["set"] = q["annotSet"]
                nearest["id"] = q["annotId"]
                nearest["index"] = index
                nearest["type"] = q["type"]
            index+=1
        if len(knockout) > 0:
            knockout.pop(nearest["index"])
            meString = (docId + "\t" + str(nearest["set"]) + "\tMeasuredEntity\t" + str(me["start"]) + "\t" + 
                        str(me["end"]) + "\t" + str(nearest["id"]+1) + "\t" + me["text"] + "\t{\"HasQuantity\": \"" + 
                        str(["id"]) + "\"}" )   
            mestrings.append(meString)
            mes.append({"annotSet": nearest["set"], "annotId": nearest["id"]+1, "start": me["start"], 
                        "end": me["end"], "text": me["text"], "type": "MeasuredEntity"})
            
    #Finally, let's process our Qualifiers:
    kitchensink = [x for x in itertools.chain(quants, mps, mes)]
    qualstrings = []
    for qual in allents['Qualifier']:
        start = qual[1]
        end = qual[2]
        nearest = {"dist": 100000000, "set": 0, "id": 0, "index": 100000000}
        index = 0
        for q in kitchensink:
            dists = [abs(start-q["start"]), abs(end-q["start"]), abs(start-q["end"]), abs(end-q["end"])]
            mindist = min(dists)
            if mindist < nearest["dist"]:
                nearest["dist"] = mindist
                nearest["set"] = q["annotSet"]
                nearest["id"] = q["annotId"]
                nearest["index"] = index
            index+=1
        if kitchensink:
          kitchensink.pop(nearest["index"])

        qualString = (docId + "\t" + str(nearest["set"]) + "\tQualifier\t" + str(qual[1]) + "\t" + 
                    str(qual[2]) + "\t" + str(nearest["id"]+1) + "\t" + qual[0] + "\t{\"Qualifies\": \"" + 
                    str(nearest["id"]) + "\"}" )
        qualstrings.append(qualString)

    # Finally, we collect everything:

    import itertools
    allstrings = [x for x in itertools.chain(quantstrings, mpstrings, mestrings, qualstrings)]
    sortedstrings = {}

    sub = open(subdir+docId + ".tsv", "w")

    for string in allstrings:
        annotSet = string.split("\t")[1]
        annotId = string.split("\t")[5]
        if annotSet not in sortedstrings:
            sortedstrings[annotSet] = {}
        sortedstrings[annotSet][annotId] = string   
    sub.write(header+"\n")
    for aset, val in sortedstrings.items():
        for aid, string in val.items():
            sub.write(string+"\n")
    sub.close()

### Exporting the data in the cuurent directory or to download in local storage -

In [47]:
import os, re, os.path
mypath = "/content/MeasEval/baseline-simpler-split"
paths = ['S0019103512003533-5300.tsv', 'S2213671113000738-684.tsv', 'S0301010413004096-693.tsv', 'S175058361300203X-1638.tsv', 'S1873506113001116-1204.tsv', 'S0006322312001096-1194.tsv', 'S0038071713001971-1388.tsv', 'S0925443913001385-1638.tsv', 'S0378383911001669-2205.tsv', 'S1873506113001116-1369.tsv', 'S175058361300203X-1240.tsv', 'S0032063312003054-2458.tsv', 'S0927775713009606-1216.tsv', 'S0019103512003995-2681.tsv', 'S0032386113005454-1245.tsv', 'S0960148113005727-1466.tsv']
for root, dirs, files in os.walk(mypath):
    for file in paths:
      try:
        os.remove(os.path.join(root, file))
      except:
        pass

In [48]:
!tar -czvf /content/submission_files.tar.xz /content/MeasEval

Streaming output truncated to the last 5000 lines.
/content/MeasEval/data/train/tsv/S0006322312001096-1136.tsv
/content/MeasEval/data/train/tsv/S1359835X13001875-1359.tsv
/content/MeasEval/data/train/tsv/S0038071712001010-944.tsv
/content/MeasEval/data/train/tsv/S0960148113002048-3775.tsv
/content/MeasEval/data/train/tsv/S0921818113002245-859.tsv
/content/MeasEval/data/train/tsv/S0167880913001229-1304.tsv
/content/MeasEval/data/train/tsv/S0925443913001385-839.tsv
/content/MeasEval/data/train/tsv/S016412121300188X-4937.tsv
/content/MeasEval/data/train/tsv/S0927775713009606-1074.tsv
/content/MeasEval/data/train/tsv/S0960148113004989-3277.tsv
/content/MeasEval/data/train/tsv/S0038071712001010-918.tsv
/content/MeasEval/data/train/tsv/S0012821X12004384-1265.tsv
/content/MeasEval/data/train/tsv/S0927024813002961-1051.tsv
/content/MeasEval/data/train/tsv/S175058361300203X-1556.tsv
/content/MeasEval/data/train/tsv/S0960896612001022-1223.tsv
/content/MeasEval/data/train/tsv/S0031405612000728-76

In [49]:
from google.colab import files
files.download("/content/submission_files.tar.xz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>